In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import os
import sys
from enum import Enum

In [2]:
class SelectorType(Enum):
    Id = 1
    Class = 2
    Name= 3

In [3]:
import time
# 特定のテキストボックスにデータを入力する（name指定）入力後にEnterを押す
def move_textbox_by_name(driver, selector, value):
    return input_textbox_and_post(driver, selector, value, SelectorType.Name)

# 特定のテキストボックスにデータを入力する（class指定）入力後にEnterを押す
def move_textbox_by_class(driver, name, value):
    return input_textbox_and_post(driver, name, value, SelectorType.Class)

# 特定のテキストボックスにデータを入力する（class指定）入力後にEnterを押す
def move_textbox_by_id(driver, Id, value):
    driver.find_element_by_id(Id).click()
    return input_textbox_and_post(driver, Id, value, SelectorType.Id)

def input_textbox_and_post(driver, name, value, selector_type):
    start = time.time()
    if selector_type == SelectorType.Id:
        driver.find_element_by_id(name).clear()
        time.sleep(1)        
        driver.find_element_by_id(name).send_keys(value)
        time.sleep(1)
        driver.find_element_by_id(name).send_keys(Keys.ENTER)

        if time.time()-start > 5 :
            return False
    elif selector_type == SelectorType.Class:
        while driver.find_element_by_class_name(name).get_attribute('value')!=value:
            driver.find_element_by_class_name(name).clear()
            time.sleep(1)        
            driver.find_element_by_class_name(name).send_keys(value)
            time.sleep(1)
            driver.find_element_by_class_name(name).send_keys(Keys.ENTER)

            if time.time()-start > 5 :
                return False
    elif selector_type == SelectorType.Name:
        while driver.find_element_by_name(name).get_attribute('value')!=value:
            driver.find_element_by_name(name).clear()
            time.sleep(1)        
            driver.find_element_by_name(name).send_keys(value)
            time.sleep(1)
            driver.find_element_by_name(name).send_keys(Keys.ENTER)

            if time.time()-start > 5 :
                return False
    return True   
        

# 検索ボックスに文字を入力して、表示された画面そスクショした後に特定のファイル名で保存する
def screen_search_result(driver, searchtext, filename, is_bing):
    if is_bing:
        move_textbox_by_class(driver, 'b_searchbox', searchtext)
    else:
        move_textbox_by_id(driver, 'ms-searchux-input-3', searchtext)
    
    # get width and height of the page
    w = driver.execute_script("return document.body.scrollWidth;")
    h = driver.execute_script("return document.body.scrollHeight;")

    # set window size
    driver.set_window_size(w,h)

    # Get Screen Shot
    driver.save_screenshot(filename)

In [ ]:
# 検証に使うブラウザを表示して、検証用アカウントにサインインする
def init_browser():
    driver_bing = webdriver.Chrome(executable_path="./drivers/chromedriver")
    driver_o365 = webdriver.Chrome(executable_path="./drivers/chromedriver")
    
    # スクショしたいページに移動
    driver_o365.get("https://www.office.com/?auth=2")
    driver_bing.get("https://www.bing.com/work/")
    
    time.sleep(1)
    
    # ADアカウントにサインイン
    move_textbox_by_name(driver_o365, "loginfmt", 'searchuser02@jbscp.onmicrosoft.com')
    move_textbox_by_name(driver_o365, "passwd", '*****')

    move_textbox_by_name(driver_bing, "loginfmt", 'searchuser02@jbscp.onmicrosoft.com')
    move_textbox_by_name(driver_bing, "passwd", '*****')

In [24]:
# Bingで検索結果をスクショする
search_text = '検索テキスト'
screen_search_result(driver_bing, search_text, 'images/bing_images/' + search_text + '.png', True)
# Office 365で検索結果をスクショする
screen_search_result(driver_o365, search_text, 'images/o365_images/' + search_text + '.png', False)